In [1]:
## Set strain for analysis and write ftns
SI = 1

import pandas as pd, numpy as np, os, glob
from Bio import SeqIO

def dotfill(r,a,fill='.'):
    return ''.join([fill for i in range(abs(len(a)-len(r)))])

def scorealign(r,a,fill='.',verbose=True):
    score = 0
    ns = []
    for i in range(len(r))[1:-1]:
        temp = r[:i] + dotfill(r,a) + r[i:]
        c = 0
        for j,n in enumerate(a):
            if n == temp[j]:
                c = c + 1
        if c > score:
            ns = temp
            score = c
            if verbose:
                print(ns,c)
    if len(ns) == 0:
        #print('Error empty sequence\n%s\n%s'%(r,a))
        ns = r[0]+dotfill(r,a)+r[-1]
    return ns

def calcpos(seq,start,end,pad=0,fill='.'):
    c = 0;
    pos = np.arange(start,end+pad)
    poss= []
    if fill in seq:
        for n in seq:
            if n != fill:
                poss.append(pos[c])
                c = c + 1
            else:
                poss.append(pos[c])
    else:
        poss = pos
    return poss

def realign(ref,alt,verbose=False):
    if len(alt) > len(ref):
        newref = scorealign(ref,alt,verbose=verbose)
        newalt = alt
    elif len(ref) > len(alt):
        newalt = scorealign(alt,ref,verbose=verbose)
        newref = ref
    else:
        assert len(alt) == len(ref)
        newref = ref
        newalt = alt
    return newref,newalt

rdict = dict(zip(['three_prime_UTR',
                  'five_prime_UTR',
                  'CDS'],
                 [3,5,0]))

def gvtodf(gv,refseqdf,gffdf,rs='REF',fill='.',rdict=rdict):

    refs = []
    alts = []
    poss = []
    for i,j in gv.iterrows():
    
        ref = j.Ref
        alt = j.Alt.split('.')[int(j.Sample)]
        start = j.Pos
        end = start + len(ref)
    
        newref,newalt = realign(ref,alt)
        newpos = calcpos(newref,start,end)
    
        refs.append(newref)
        alts.append(newalt)
        poss.append(newpos)
    
    pos = np.concatenate(poss)
    ref = [n for n in ''.join(refs)]
    alt = [n for n in ''.join(alts)]

    df = pd.DataFrame([pos,ref,alt],
                      index=['Pos','Ref','Alt']).T
    df['Isvar'] = 1

    check = refseqdf[(refseqdf.Pos.isin(df.Pos))]
    failed = []
    for i,j in check.iterrows():
        
        tocheck = [s for s in 
                   df[(df.Pos==j.Pos)].Ref.tolist() 
                   if (s != fill)]
        if len(tocheck) > 1: ## Should be one base
            failed.append(j.Pos)
        elif not tocheck[0] == j[rs]:
            failed.append(j.Pos)
            
    if len(failed) > 0:
        print('Reference alignment failed')
    
    dfadd = refseqdf[~(refseqdf.Pos.isin(df.Pos))].copy()
    dfadd['Ref'] = dfadd[rs]
    dfadd['Alt'] = dfadd[rs]
    dfadd['Isvar'] = 0
    dfadd.drop(rs,axis=1,inplace=True)

    res = pd.concat([dfadd,df]).sort_values('Pos')
    #res['Strand'] = -1 if gffdf.Strand.min() == '-' else 1
    res['Strand'] = gffdf.Strand.min()
    res['Type'] = -1
    res['Phase'] = -1

    for i,j in gffdf[(gffdf.Type!='gene')].iterrows():
    
        nb = np.arange(j.Start,j.End+1)
        res.loc[(res.Pos.isin(nb)),'Type'] = j.Type
    
        if j.Type == 'CDS':
            res.loc[(res.Pos.isin(nb)),'Phase'] = int(j.Phase)
    
    assert -1 not in res[(res.Type=='CDS')].Phase.tolist()
    
    res.Type.replace(rdict,inplace=True)

    return res,failed

In [2]:
geno_path = '../DATA/GENOTYPE/CDx-ill-SNP-INDEL-df-104-blocked.csv.gz'
genos = pd.read_csv(geno_path)

genos['Alt'] = genos[['Ref','Alt']].apply('.'.join,axis=1)

genos['Chromosome'] = [int(c[-2:]) for c in genos.Chrom]

genos.head()

,Chrom,Pos,Qual,Callrate,Ref,Alt,Altlen,Dp,Type,Vcfix,...,SS-B565,SS-B600,SS-B574,SS-B872_cor,SS-B873_cor,SS-B360,SS-B397,SS-B564,SS-B382,Chromosome
0,Chr01,5016,104492.0,1.0,C,C.A,1,6510,snp,289,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1
1,Chr01,5299,112647.0,1.0,T,T.C,1,9711,snp,293,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1
2,Chr01,5464,112658.0,1.0,T,T.C,1,9375,snp,294,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1
3,Chr01,6120,109003.0,1.0,T,T.C,1,9311,snp,311,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1
4,Chr01,6166,114638.0,1.0,G,G.A,1,9269,snp,312,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1


In [3]:
## Bring in GFF file
gffpath = '../DATA/xl280genome.gff3.csv.gz'
gff = pd.read_csv(gffpath,index_col=0)

descriptions = ['hypothetical protein',
                'unspecified product',
                'conserved hypothetical protein']
foi = ['gene','three_prime_UTR','five_prime_UTR','CDS']


gff.columns = [c.capitalize() for c in gff.columns]

gff['Seqid'] = gff['Contig']
gff['Strand'] = gff['Strand'].replace(dict(zip(['-','+'],[-1,1])))
gff['Parent'] = [a.split('Parent=')[-1].split(';')[0].split('ID=')[-1] 
                 for a in gff.Attribute ]

genes = gff[(gff.Type=='gene')].sort_values('Start').copy()

genes['Description'] = [a.split('description=')[-1].split('%2C')[0] 
                        for a in genes.Attribute]

genes.head()

,Type,Gene,Start,End,Strand,Contig,Phase,Attribute,Description,Id,Seqid,Parent
4012,gene,CNE00010,38,910,-1,Chr05,.,ID=CNE00010;description=hypothetical protein,hypothetical protein,CNE00010,Chr05,CNE00010
67409,gene,CNH03875,76,2412,-1,Chr08,.,ID=CNH03875;description=hypothetical protein,hypothetical protein,CNH03875,Chr08,CNH03875
34312,gene,CNJ00010,171,4804,1,Chr10,.,ID=CNJ00010;description=conserved hypothetical...,conserved hypothetical protein,CNJ00010,Chr10,CNJ00010
70098,gene,CNG00010,196,1723,1,Chr07,.,ID=CNG00010;description=hypothetical protein,hypothetical protein,CNG00010,Chr07,CNG00010
70249,gene,CNH03660,211,2748,-1,Chr12,.,ID=CNH03660;description=alpha-amylase AmyA%2C ...,alpha-amylase AmyA,CNH03660,Chr12,CNH03660


In [4]:
## Bring in reference file
refpath = '/home/croth/xl280genome.fasta'

REF = [s for s in SeqIO.parse(refpath,format='fasta')]
len(REF)

14

In [5]:
## How many are in ric 8
ric8_name = 'CNN01270-t26_1'
gff[(gff.Parent==ric8_name)].shape

(18, 12)

In [6]:
## Collect known genes
known = genes[~(genes.Description.isin(descriptions))]
known_genes = known.Gene.tolist()
known.shape

(3423, 12)

In [7]:
## Collect unknown genes
unknown = genes[(genes.Description.isin(descriptions))]
unknown_genes = unknown.Gene.tolist()
unknown.shape

(2577, 12)

In [32]:
maxqtl_info = pd.read_csv('../DATA/Max_QTL_bounds.csv',index_col=0)
maxqtl_info.head()

,Cumpos,Chrom,Max,Pos
522,3142153,Chr02,5.958631,847396
523,3143506,Chr02,6.297631,848749
524,3144731,Chr02,6.466895,849974
525,3205392,Chr02,5.130648,910635
527,3209641,Chr02,5.192451,914884


In [33]:
maxqtl_info['Genoix'] = [genos[(genos.Chrom==mr.Chrom) & 
                               (genos.Pos==mr.Pos)].index.max() 
                                 for i,mr in maxqtl_info.iterrows()]

In [37]:
qtl_bounds = []
for i,cb in maxqtl_info.groupby('Chrom'):
    
    l = genos[(genos.Block == genos.loc[cb.Genoix.min(),'Block'])].Pos.min()
    r = genos[(genos.Block == genos.loc[cb.Genoix.max(),'Block'])].Pos.max()
    
    qtlb = (i,l,r)
    qtl_bounds.append(qtlb)
    
qtl_bounds = pd.DataFrame(qtl_bounds,columns=['Chrom','Left','Right'])
qtl_bounds

,Chrom,Left,Right
0,Chr02,847396,1001789
1,Chr11,37942,98737
2,Chr12,552849,616400
3,Chr14,353726,423207


In [38]:
ekb = 0

In [39]:
qtlgenes = np.concatenate([genes[(genes.Contig ==c.Chrom) & 
              (genes.Start>=c.Left-ekb) & 
              (genes.End <= c.Right+ekb)].Parent.tolist() 
            for i,c in qtl_bounds.iterrows()])
len(qtlgenes)

136

In [40]:
## Gather parents
parents = gff[(gff.Type.isin(foi)) & 
              (gff.Type!='gene') & 
              (gff.Gene.isin(qtlgenes))
             ][['Gene','Parent']
              ].drop_duplicates()
parents.head()

,Gene,Parent
819,CNK00300,CNK00300-t26_1
1588,CNN01300,CNN01300-t26_1
1777,CNB03090,CNB03090-t26_1
2361,CNK00280,CNK00280-t26_1
3560,CNL05630,CNL05630-t26_1


In [41]:
salt_QTL = (952008, 994759)
saltgenes = genes[(genes.Contig =='Chr10') & 
              (genes.Start>= np.min(salt_QTL) - ekb) & 
              (genes.End <= np.max(salt_QTL) + ekb)].Parent.tolist() 
len(saltgenes)

16

In [42]:
saltparents  = gff[(gff.Type.isin(foi)) & 
              (gff.Type!='gene') & 
              (gff.Gene.isin(saltgenes))
             ][['Gene','Parent']
              ].drop_duplicates()
saltparents.head()

,Gene,Parent
5046,CNJ03153,CNJ03153-t26_1
15762,CNJ03173,CNJ03173-t26_1
34951,CNJ03090,CNJ03090-t26_1
36240,CNJ03120,CNJ03120-t26_1
38621,CNJ03175,CNJ03175-t26_1


In [43]:
verbos = False

In [44]:
ric8_name in parents.Parent.tolist()

True

In [45]:
known_parents = parents.Parent.tolist()

In [46]:
Fo = ['SS-A837','XL280a','XL280alpha']
genos[Fo].head()

,SS-A837,XL280a,XL280alpha
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [47]:
samplename = 'SS-A837'

In [48]:
savepath = '../DATA/GENOTYPE/GENES/%s/'%samplename
if not os.path.exists(savepath):
    os.mkdir(savepath)
savepath

'../DATA/GENOTYPE/GENES/SS-A837/'

In [49]:
info_cols = [c for c in genos.columns if c[:2]!='SS' and c not in Fo]
len(info_cols)

22

In [50]:
run_parents = parents.Parent.tolist() + saltparents.Parent.tolist()

In [51]:
## Is the GFF zero based?
zb = True

In [52]:
novars = []
notype = []
for ssk1name in run_parents:
#for ssk1name in known_genes:
    genesave = savepath+ssk1name+'.csv.gz'
    #if os.path.exists(genesave):
    #    continue
    ## Take gene info from gff file
    ssk1df = gff[(gff.Type.isin(foi)) & 
                 (gff.Parent==ssk1name)
                ].sort_values('Start').copy()
    ssk1df['Start'] = ssk1df['Start'] + (1 if zb else 0)
    
    assert len(ssk1df.Strand.unique())==1
    if len(ssk1df.Type.unique())!=3:
        notype.append(ssk1name+'\n')
        continue
    ## For a sample of interest bring in data
    ##sample = pd.read_csv(samplepath,index_col=0)
    sample = genos.loc[genos[samplename].dropna().index,
                       info_cols+[samplename]]
    
    sample['Sample'] = sample[samplename]
    ## Locate genetic variants within a gene
    gv = sample[(sample.Chrom==ssk1df.Seqid.min()) & 
            (sample.Pos>=ssk1df.Start.min()) & 
            (sample.Pos<=ssk1df.End.max())]
    if gv.shape[0] == 0:
        novars.append(ssk1name+'\n')
        if verbos:
            print('No variants within %s for strain %s\n'%(
            ssk1name,samplename))
        continue
    ## From the referecne take the sequence of interest
    gene_seq = [s.seq[ssk1df.Start.min()-(0 if zb else 1):ssk1df.End.max()] 
            for s in REF if s.id == ssk1df.Seqid.min()][0]
    ## Make a dataframe from these sequences
    refseqdf = pd.DataFrame([np.arange(ssk1df.Start.min()+(1 if zb else 0),
                                       ssk1df.End.max()+1),
                     [n for n in str(gene_seq)]],
                            index=['Pos','REF']).T;
    ## Make gene variant dataframe
    res,failed  = gvtodf(gv,refseqdf,ssk1df)
    res['Gene'] = ssk1name
    res['Sample'] = samplename
    ## Save and print results
    res.to_csv(genesave)
    if verbos:
        print('%s\n-------------\n%s\n%s\n-------------\n%s\n'%(
            ssk1name,res.Pos.unique().shape[0],res.shape[0],
            res[(res.Isvar==1)].shape[0]))

In [53]:
novars_path = savepath+'NOVARS.csv'
if os.path.exists(novars_path):
    os.remove(novars_path)
    
notype_path = savepath+'NOTYPE.csv'
if os.path.exists(notype_path):
    os.remove(notype_path)

In [54]:
open(novars_path,'w').writelines(novars)
open(notype_path,'w').writelines(notype)

In [55]:
len(novars)

10

In [56]:
len(notype)

12

In [57]:
len(run_parents) - (len(novars) + len(notype))

123

In [58]:
%%bash
cd ../DATA/GENOTYPE/GENES/SS-A837/
ls -l *csv.gz | wc -l

123
